# 3 - Population Data

Extract population data from IBGE (Instituto Brasileiro de Geografia e Estatística).

These data are only available annually (whereas natality data is available monthly). The data for 2015-2021 are projections from the census before last (in 2010). The data for 2022 is a preview for the last census (in 2022).

In [16]:
import os
import re
import pandas as pd

## Experiment with one set of population data

Let's try out the transformations so we can make sure it'll go smoothly when we loop through all the population data.

In [77]:
# read the file
popdf = pd.read_excel(r'../data/external/POP2021_20221212.xls', sheet_name = 'BRASIL E UFs', skiprows = 1)
popdf

,BRASIL E UNIDADES DA FEDERAÇÃO,Unnamed: 1,POPULAÇÃO ESTIMADA
0,Brasil,NaN,213317639
1,Região Norte,NaN,18906962
2,Rondônia,NaN,1815278
3,Acre,NaN,906876
4,Amazonas,NaN,4269995
5,Roraima,NaN,652713
6,Pará,NaN,8777124
7,Amapá,NaN,877613
8,Tocantins,NaN,1607363
9,Região Nordeste,NaN,57667842


In [78]:
# drop bogus column
popdf = popdf.drop(columns = 'Unnamed: 1')
popdf

,BRASIL E UNIDADES DA FEDERAÇÃO,POPULAÇÃO ESTIMADA
0,Brasil,213317639
1,Região Norte,18906962
2,Rondônia,1815278
3,Acre,906876
4,Amazonas,4269995
5,Roraima,652713
6,Pará,8777124
7,Amapá,877613
8,Tocantins,1607363
9,Região Nordeste,57667842


In [80]:
# drop any rows containing NaN. This is the easiest way to eliminate the main table footer
popdf = popdf.dropna()
popdf

,BRASIL E UNIDADES DA FEDERAÇÃO,POPULAÇÃO ESTIMADA
0,Brasil,213317639
1,Região Norte,18906962
2,Rondônia,1815278
3,Acre,906876
4,Amazonas,4269995
5,Roraima,652713
6,Pará,8777124
7,Amapá,877613
8,Tocantins,1607363
9,Região Nordeste,57667842


In [83]:
# rename the columns
popdf.columns = 'state', 'population'
popdf

,state,population
0,Brasil,213317639
1,Região Norte,18906962
2,Rondônia,1815278
3,Acre,906876
4,Amazonas,4269995
5,Roraima,652713
6,Pará,8777124
7,Amapá,877613
8,Tocantins,1607363
9,Região Nordeste,57667842


In [84]:
# retype the 'population' column to int.
# it won't go well because some cells have footnotes (directly in them, wtf?)
popdf['population'] = popdf['population'].astype(int)
popdf

ValueError: invalid literal for int() with base 10: '3.289.290(1)'

In [85]:
# we then must resort to regex to parse the numbers out

popdf['population'] = popdf['population'].astype('str').str.strip().str.replace(
    r'((?:\d?\d?\d\.?)*\d?\d?\d)\s?\(\d+\)', r'\1',
    regex = True
).str.replace('.','', regex = False).astype(int)

popdf

,state,population
0,Brasil,213317639
1,Região Norte,18906962
2,Rondônia,1815278
3,Acre,906876
4,Amazonas,4269995
5,Roraima,652713
6,Pará,8777124
7,Amapá,877613
8,Tocantins,1607363
9,Região Nordeste,57667842


In [86]:
# we only want state populations, not aggregations by regions

regions_mask = popdf['state'].str.contains(r'((B|b)rasil|(R|r)egi)')
popdf = (popdf
    .drop(popdf.index[regions_mask])
    .reset_index(drop = True)
)
popdf

C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\2479675291.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = popdf['state'].str.contains(r'((B|b)rasil|(R|r)egi)')


,state,population
0,Rondônia,1815278
1,Acre,906876
2,Amazonas,4269995
3,Roraima,652713
4,Pará,8777124
5,Amapá,877613
6,Tocantins,1607363
7,Maranhão,7153262
8,Piauí,3289290
9,Ceará,9240580


In [87]:
popdf['year'] = 2022
popdf

,state,population,year
0,Rondônia,1815278,2022
1,Acre,906876,2022
2,Amazonas,4269995,2022
3,Roraima,652713,2022
4,Pará,8777124,2022
5,Amapá,877613,2022
6,Tocantins,1607363,2022
7,Maranhão,7153262,2022
8,Piauí,3289290,2022
9,Ceará,9240580,2022


## Main Loop

Now let's extract all population data. It only makes sense to extract population data 2015-present, since that is how far back the natality data goes.

In [101]:
# define processing steps
def process_drop_unused_cols(df):
    df = df.drop(columns = 'Unnamed: 1')
    return df

def process_rename_cols(df):
    df.columns = 'state', 'population'
    return df

def process_change_type(df):
    try:
        df['population'] = df['population'].astype(int)
    except ValueError:  # invalid format for auto converting string to int
        regex_pat = r'((?:\d?\d?\d\.?)*\d?\d?\d)\s?\(.+\)'
        
        df['population'] = df['population'].astype('str').str.strip().str.replace(
            regex_pat, r'\1',
            regex = True
        ).str.replace('.','', regex = False)
        df['population'] = df['population'].astype(int)
    print(df)
    return df

def process_remove_regions(df):
    regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)
    df = (df
        .drop(df.index[regions_mask])
        .reset_index(drop = True)
    )
    
    return df

def process_add_year(df, year):
    df['year'] = year
    return df

def process_reorder_cols(df):
    new_order = ['year', 'state', 'population']
    df = df.reindex(columns = new_order)
    return df

def process_df(df_raw, year):
    df_raw = df_raw.copy()

    df = (df_raw
        .pipe(process_drop_unused_cols)
        .dropna()
        .pipe(process_rename_cols)
        .pipe(process_change_type)
        .pipe(process_remove_regions)
        .pipe(process_add_year, year)
        .pipe(process_reorder_cols)
    )

    return df

In [93]:
# config for Excel extraction

pop_raw_data = {
    2015: {
        'file': r'estimativa_TCU_2015_20170614.xls',
        'sheet': 'BRASIL_E_UFs'
    },
    2016: {
        'file': r'estimativa_TCU_2016_20170614.xls',
        'sheet': 'BRASIL E UFs'
    },
    2017: {
        'file': r'POP2017_20220905.xls',
        'sheet': 'BRASIL_E_UFs'
    },
    2018: {
        'file': r'POP2018_20220905.xls',
        'sheet': 'BRASIL E UFs'
    },
    2019: {
        'file': r'POP2019_20220905.xls',
        'sheet': 'BRASIL E UFs'
    },
    2020: {
        'file': r'POP2020_20220905.xls',
        'sheet': 'BRASIL E UFs'
    },
    2021: {
        'file': r'POP2021_20221212.xls',
        'sheet': 'BRASIL E UFs'
    },
    2022: {
        'file': r'POP2022_Brasil_e_UFs.xls',
        'sheet': 'BRASIL E UFs'
    },
}

In [103]:
# main loop for concatenating all years

path = r'../data/external'

# dataframe accumulator
popdf = None

for year, data in pop_raw_data.items():
    fn = data['file']
    sheet_name = data['sheet']
    
    abspath = os.path.abspath(os.path.join(path, fn))
    popdf_raw = pd.read_excel(abspath, sheet_name = sheet_name, skiprows = 1)
    popdf_single = process_df(popdf_raw, year)

    
    if popdf is None: # first batch of data
        popdf = popdf_single
    else: # accumulator already has some data in it
        popdf = pd.concat([popdf, popdf_single])

popdf = (popdf
    .sort_values(by = ['year', 'state'])
    .reset_index(drop = True)
)

popdf

                  state  population
0                Brasil   204450649
1          Região Norte    17472636
2              Rondônia     1768204
3                  Acre      803513
4              Amazonas     3938336
5               Roraima      505665
6                  Pará     8175113
7                 Amapá      766679
8             Tocantins     1515126
9       Região Nordeste    56560081
10             Maranhão     6904241
11                Piauí     3204028
12                Ceará     8904459
13  Rio Grande do Norte     3442175
14              Paraíba     3972202
15           Pernambuco     9345173
16              Alagoas     3340932
17              Sergipe     2242937
18                Bahia    15203934
19       Região Sudeste    85745520
20         Minas Gerais    20869101
21       Espírito Santo     3929911
22       Rio de Janeiro    16550024
23            São Paulo    44396484
24           Região Sul    29230180
25               Paraná    11163018
26       Santa Catarina     

C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)
C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)
C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)


                  state  population
0                Brasil   208494900
1          Região Norte    18182253
2              Rondônia     1757589
3                  Acre      869265
4              Amazonas     4080611
5               Roraima      576568
6                  Pará     8513497
7                 Amapá      829494
8             Tocantins     1555229
9       Região Nordeste    56760780
10             Maranhão     7035055
11                Piauí     3264531
12                Ceará     9075649
13  Rio Grande do Norte     3479010
14              Paraíba     3996496
15           Pernambuco     9496294
16              Alagoas     3322820
17              Sergipe     2278308
18                Bahia    14812617
19       Região Sudeste    87711946
20         Minas Gerais    21040662
21       Espírito Santo     3972388
22       Rio de Janeiro    17159960
23            São Paulo    45538936
24           Região Sul    29754036
25               Paraná    11348937
26       Santa Catarina     

C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)
C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)
C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)


                  state  population
0                Brasil   213317639
1          Região Norte    18906962
2              Rondônia     1815278
3                  Acre      906876
4              Amazonas     4269995
5               Roraima      652713
6                  Pará     8777124
7                 Amapá      877613
8             Tocantins     1607363
9       Região Nordeste    57667842
10             Maranhão     7153262
11                Piauí     3289290
12                Ceará     9240580
13  Rio Grande do Norte     3560903
14              Paraíba     4059905
15           Pernambuco     9674793
16              Alagoas     3365351
17              Sergipe     2338474
18                Bahia    14985284
19       Região Sudeste    89632912
20         Minas Gerais    21411923
21       Espírito Santo     4108508
22       Rio de Janeiro    17463349
23            São Paulo    46649132
24           Região Sul    30402587
25               Paraná    11597484
26       Santa Catarina     

C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)
C:\Users\bdzp\AppData\Local\Temp\ipykernel_29304\1694074581.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  regions_mask = df['state'].str.contains(r'((B|b)rasil|(R|r)egi)', regex = True)


,year,state,population
0,2015,Acre,803513
1,2015,Alagoas,3340932
2,2015,Amapá,766679
3,2015,Amazonas,3938336
4,2015,Bahia,15203934
...,...,...,...
211,2022,Roraima,634805
212,2022,Santa Catarina,7762154
213,2022,Sergipe,2211868
214,2022,São Paulo,46024937


In [104]:
popdf[popdf['state'].str.contains('aulo')]

,year,state,population
25,2015,São Paulo,44396484
52,2016,São Paulo,44749699
79,2017,São Paulo,45094866
106,2018,São Paulo,45538936
133,2019,São Paulo,45919049
160,2020,São Paulo,46289333
187,2021,São Paulo,46649132
214,2022,São Paulo,46024937


In [105]:
# save processed dataframe to parquet file
path = r'../data/processed'
fn = r'brazil_population_state_annually.parquet'

abspath = os.path.abspath(os.path.join(path, fn))

popdf.to_parquet(abspath)